In [1]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras import backend as K
#import cudf (#only works if GPU is on)
import gc
import os
import polars as pl
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

2024-05-08 11:54:01.147846: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 11:54:01.147975: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 11:54:01.301499: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Dataset: https://www.kaggle.com/datasets/titericz/leap-dataset-giba
# This dataset was created using all train and test data.
# Train is 10M samples and was split in 17 parquet files. Test is a single parquet file.
from glob import glob
train_files = sorted(glob("/kaggle/input/leap-dataset-giba/train_batch/*.parquet"))
test_files = glob("/kaggle/input/leap-dataset-giba/test_batch/*.parquet")
len(train_files), len(test_files)

(17, 1)

In [3]:
# Train on 2/17 of the full dataset
train = pd.read_parquet(train_files[:2]).astype('float32')
#train = cudf.from_pandas(train) # Send to GPU for speedup
gc.collect()

# Validate on last parquet file (625000 samples)
valid = pd.read_parquet(train_files[-1]).astype('float32')
#valid = cudf.from_pandas(valid) # Send to GPU for speedup
gc.collect()

test  = pd.read_parquet(test_files[0]).astype('float32')
#test = cudf.from_pandas(test) # Send to GPU for speedup
gc.collect()

train.shape, valid.shape, test.shape

((1250000, 859), (625000, 859), (625000, 491))

In [4]:
#The end of feature columns
print(f'Feature col ends here {train.columns[490]}') 
#The start of target
print(f'Target col starts here {train.columns[491]}')

Feature col ends here pbuf_N2O_26
Target col starts here ptend_t_0


In [5]:
feat_col = train.columns[:491]
target = train.columns[491:]
feat_col, target

(Index(['sample_id', 'state_t_0', 'state_t_1', 'state_t_2', 'state_t_3',
        'state_t_4', 'state_t_5', 'state_t_6', 'state_t_7', 'state_t_8',
        ...
        'pbuf_N2O_17', 'pbuf_N2O_18', 'pbuf_N2O_19', 'pbuf_N2O_20',
        'pbuf_N2O_21', 'pbuf_N2O_22', 'pbuf_N2O_23', 'pbuf_N2O_24',
        'pbuf_N2O_25', 'pbuf_N2O_26'],
       dtype='object', length=491),
 Index(['ptend_t_0', 'ptend_t_1', 'ptend_t_2', 'ptend_t_3', 'ptend_t_4',
        'ptend_t_5', 'ptend_t_6', 'ptend_t_7', 'ptend_t_8', 'ptend_t_9',
        ...
        'ptend_v_58', 'ptend_v_59', 'cam_out_NETSW', 'cam_out_FLWDS',
        'cam_out_PRECSC', 'cam_out_PRECC', 'cam_out_SOLS', 'cam_out_SOLL',
        'cam_out_SOLSD', 'cam_out_SOLLD'],
       dtype='object', length=368))

In [6]:
train_feat = train[feat_col]
train_target = train[target]
valid_feat = valid[feat_col]
valid_target = valid[target]

In [7]:
#X_train = train.select(feat_col).to_numpy()
#y = train_df.select(TARGET_COLS).to_numpy()
X_train = train_feat.to_numpy()
y_train = train_target.to_numpy()
X_val = valid_feat.to_numpy()
y_val = valid_target.to_numpy()
test_feats = test.to_numpy()

X_train.shape, y_train.shape, X_val.shape, y_val.shape, test_feats.shape

((1250000, 491), (1250000, 368), (625000, 491), (625000, 368), (625000, 491))

In [8]:
# Save data to files
np.save('clsm_Xtrain', X_train)
np.save('clsm_ytrain', y_train)
np.save('clsm_Xval', X_val)
np.save('clsm_yval', y_val)
np.save('clsm_test_feats', test_feats)